# Dataset: Abnormal blood cells

In [ ]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

In [ ]:
df = pd.read_pickle('BL_RL_N_malignanCells.pkl').dropna()
df.head()

## Análisis exploratorio

In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
#variables que describen celula 
cel_info = df[['Area_cel','EquivDiameter_cel','Eccentricity_cel','Perimeter_cel','Solidity_cel','Extent_cel','circularity_cel','Elongation_cel','roundnessCH_cel','convexity_cel','circleVariance_cel','ellipVariance_cel']]
#correlacion
plt.subplots(figsize=(20,15))
sns.heatmap(cel_info.corr(), annot= True );

In [ ]:
cel_info_red = df[['Perimeter_cel','Solidity_cel','Extent_cel','circularity_cel','Elongation_cel','roundnessCH_cel','circleVariance_cel']]#son las que mustran mayor correlacion

sns.pairplot(cel_info_red);

In [ ]:
#variables que describen nucleo
nuc_info = df[['Area_nuc','EquivDiameter_nuc','Eccentricity_nuc','Perimeter_nuc','Solidity_nuc','Extent_nuc','circularity_nuc','Elongation_nuc','roundnessCH_nuc','convexity_nuc','circleVariance_nuc','ellipVariance_nuc']]
plt.subplots(figsize=(20,15))
#correlacion
sns.heatmap(nuc_info.corr(), annot= True );

In [ ]:
nuc_info_red = df[['Perimeter_nuc','Solidity_nuc','Extent_nuc','circularity_nuc','Elongation_nuc','roundnessCH_nuc','circleVariance_nuc']]#las que muestran mayor correlacion

sns.pairplot(nuc_info_red);

In [ ]:
cel_info_red.hist(figsize = (20,20));

In [ ]:
nuc_info_red.hist(figsize = (20,20));

In [ ]:
X = df.drop(columns=['tipoCelula','entidad','historia','frotis','archivos'])
y = df['tipoCelula']
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size= 0.3, random_state=43)

In [ ]:
glm = LogisticRegression().fit(X_train,y_train)
y_pred = glm.predict(X_test)

In [ ]:
CM = confusion_matrix(y_pred,y_test,labels=['BL','RL','N'])
CM = np.resize(CM,new_shape=[3,3])
CM = CM/np.sum(CM,axis=1) 
CM = pd.DataFrame(CM, index = ['BL','RL','N'], columns=['BL','RL','N'])
sns.heatmap(CM,annot=True,cmap = 'YlGnBu')
plt.tight_layout()
plt.show()